# Vaja 2: Go App

- včasih velike aplikacije v enem kosu, veliko časa potrebnega za release
- zdaj pa želimo imeti aplikacije na več neodvisnih manjših koščkov in hitre release
- miscroservices -> hitrejši razvoj
- kako se je spreminjalo od zgodovine do danes

## Priprava

Prenesemo gradiva z Git-a in se premaknemo v mapo `02_vaja_go_app`

## Del 1:  Primer monolitske aplikacije

Se premaknemo v mapo `part01_monolith/monolith`

    cd part01_monolith/

Pogledamo `monolith/main.go`

Run the monolith container and get it's IP:

    CID=$(docker run -d leon11sj/icta-monolith:v1)
    echo $CID

    docker ps



List the monolith image:

    docker images leon11sj/icta-monolith:v1



    docker inspect $CID

    CIP=$(docker inspect --format '{{ .NetworkSettings.IPAddress }}' ${CID})
    echo $CIP

Test the container
    
    curl $CIP

**Prikaz delovanja**

Verzija:

    curl $CIP/version

Testiramo secure:

    curl $CIP/secure

Authenticate (password is password):
    
    curl $CIP/login -u user

It prints out the token.

You can copy and paste the long token in to the next command manually, but copying long, wrapped lines in cloud shell is broken. To work around this, you can either copy the JWT token in pieces, or - more easily - by assigning the token to a shell variable as follows

> `sudo apt  install jq` v primeru da ni nameščeno

    TOKEN=$(curl $CIP/login -u user | jq -r '.token')
    echo $TOKEN

Access the secure endpoint using the JWT:
    
    curl -H "Authorization: Bearer $TOKEN" $CIP/secure

**Dodajanje slike na lasten DockerHub**

Add your own tag:

    docker tag leon11sj/icta-monolith:v1 <your username>/icta-monolith:v1
    sudo docker login
    sudo docker push <your username>/icta-monolith:v1

**Pospravljanje**

Ugasnemo kontejner in zbrišemo:

    docker rm -f $CID
    docker image rm leon11sj/icta-monolith:v1

### Namestitev na Kubernetes

```yml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: monolith
  labels:
    app: monolith
spec:
  replicas: 2
  selector:
    matchLabels:
      app: monolith
  template:
    metadata:
      labels:
        app: monolith
    spec:  
      containers:
        - name: monolith
          image: leon11sj/icta-monolith:v1
          args:
            - "-http=0.0.0.0:80"
            - "-health=0.0.0.0:81"
            - "-secret=secret"
          ports:
            - name: http
              containerPort: 80
            - name: health
              containerPort: 81
          resources:
            limits:
              cpu: 0.2
              memory: "10Mi"
          livenessProbe:
            httpGet:
              path: /healthz
              port: 81
              scheme: HTTP
            initialDelaySeconds: 5
            periodSeconds: 15
            timeoutSeconds: 5
          readinessProbe:
            httpGet:
              path: /readiness
              port: 81
              scheme: HTTP
            initialDelaySeconds: 5
            timeoutSeconds: 1
                
                ```

Namestimo monolith aplikacijo:    
    
    kubectl apply -f kubernetes/monolith-deployment.yaml

    kubectl get pods
    kubectl get deployments monolith
    kubectl describe deployments monolith

#### Preverimo delovanje

**Cloud shell 1: set up port-forwarding**

    kubectl port-forward <POD_NAME> 8080:80

**Open new Cloud Shell session 2**

    curl http://127.0.0.1:8080

    curl http://127.0.0.1:8080/secure

**Cloud shell 2 - log in**

    curl -u user http://127.0.0.1:8080/login

    curl -H "Authorization: Bearer <token>" http://127.0.0.1:8080/secure

**View logs**

    kubectl logs <POD_NAME>
    kubectl logs -f <POD_NAME>


### Dodamo LoadBalancer

```yml
apiVersion: v1
kind: Service
metadata:
  name: monolith
spec:
  type: LoadBalancer
  selector:
    app: monolith
  ports:
  - port: 80
    targetPort: 80

```

    kubectl apply -f kubernetes/monolith-service.yml
    kubectl get svc

#### Preverimo delovanje

    curl http://35.246.197.19

### Odstranimo vse

    kubectl delete svc monolith
    kubectl delete deployment monolith

## Primer mikrostoritvene aplikacije

Glavne prednosti:
- modularne
- enostaven deploy
- samostojna skalabilnost

### Ustvarimo konfiguracijo

Next we need to create a configmap entry for the proxy.conf nginx configuration file using the kubectl create configmap command:

    kubectl create configmap nginx-conf --from-file=conf/frontend.conf

Use the kubectl describe configmap command to get more details about the nginx-proxy-conf configmap entry:

    kubectl describe configmap nginx-conf


### Namestimo deployments

    kubectl apply -f kubernetes/deployments/hello.yaml
    kubectl apply -f kubernetes/deployments/auth.yaml

Namestimo services:

    kubectl apply -f kubernetes/services/auth.yaml
    kubectl apply -f kubernetes/services/hello.yaml

Namestimo NGINX:

    kubectl apply -f kubernetes/deployments/frontend.yaml

Namestimo LoadBlanacer:

    kubectl apply -f kubernetes/services/frontend.yaml

**Preverimo delovanje**

    kubectl get svc
    export CIP=35.246.197.19
    echo $CIP
    



```bash
curl $CIP/version
curl $CIP/secure
TOKEN=$(curl $CIP/login -u user | jq -r '.token')
echo $TOKEN    
curl -H "Authorization: Bearer $TOKEN" $CIP/secure
    
```

**Odstranimo vse**

    kubectl delete svc frontend
    kubectl delete svc auth
    kubectl delete svc hello
    
    kubectl delete deployments frontend auth hello
    
    kubectl delete configmaps nginx-conf